The following script pre-process the data for washing machines used in Appliance Manager and Appliance Comparision.

In [1]:
import pandas as pd #import libraries

In [2]:
washing_machines = pd.read_csv('./tmp9ny0pl_cw_2019_04_11.csv') #store csv in a dataframe

In [4]:
washing_machines = washing_machines[['Brand', 'Model No', 'Cap', 'Country', 'CEC_', 'New Star']] #remove unwanted columns

In [5]:
washing_machines.columns = ['Brand Name','Model Number', 'Capacity','Manufacturing Countries', 'Comparative Energy Consumption','Star Rating'] #renaming columns

In [6]:
washing_machines['Appliance Type'] = 'Washing Machines' #adding column for appliance type

In [7]:
wm = washing_machines

In [8]:
wm["Brand Name"] = wm["Brand Name"].str.replace('SAMSUNG ELECTRONICS','SAMSUNG') #replacing similar values

In [9]:
wm['Brand Name'] = wm['Brand Name'].str.upper() #renaming brands to upper case

In [10]:
c = wm

In [11]:
#splitting multiple values into separate rows
c.rename(columns={'Model Number': 'var1'}, inplace=True)

c = \
(c.set_index(c.columns.drop('var1',1).tolist())
   .var1.str.split(',', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'var1'})
   .loc[:, c.columns]
)

c.var1 = c.var1.str.lstrip() #removing excess space

c.drop_duplicates(subset='var1', keep="first", inplace=True) #removing duplicate values

c.rename(columns={'var1': 'Model Number'}, inplace=True) #renaming column


In [12]:
#
c.loc[c['Comparative Energy Consumption'] < 400, 'EcoLife Rating'] = 'Low' 
#c.loc[c['Comparative Energy Consumption'] > 225 & c['Comparative Energy Consumption'] < 300, 'EcoLife Rating'] = 'Medium' 
c.loc[c['Comparative Energy Consumption'] > 650, 'EcoLife Rating'] = 'High' 

In [13]:
c['EcoLife Rating'] = c['EcoLife Rating'].fillna('Medium')

In [14]:
c.sample(5)

Brand Name Model Number  Capacity Manufacturing Countries  \
417            MIDEA       DMWM95       9.5                   China   
176          SAMSUNG  WW80J4413IW       8.0          China,Thailand   
389          SIMPSON     SWT8063E       8.0                Thailand   
12         WHIRLPOOL    WTWE75AW1       7.5                   India   
280  FISHER & PAYKEL     WA8560E1       8.5                Thailand   

     Comparative Energy Consumption  Star Rating    Appliance Type  \
417                             762          2.0  Washing Machines   
176                             290          4.0  Washing Machines   
389                             460          3.0  Washing Machines   
12                              620          2.0  Washing Machines   
280                             313          4.0  Washing Machines   

    EcoLife Rating  
417           High  
176            Low  
389         Medium  
12          Medium  
280            Low

In [15]:
c.to_csv('washing machines.csv', sep=',', encoding='utf-8')